# Excercise 07: Fault Tolerance



## Combiner

To combine the results from multiple models, you can use the Seldon concept of combiners. Combiners aggregate the result of child models to a single response (check https://docs.seldon.io/projects/seldon-core/en/v1.1.0/python/python_component.html#combiners)

You find our combiner in the folder `services/support/seldon-combiner`. Study its source code. When does it return 1?

Create a new file in the folder pipelines named `combiners-seldon-model.yml`:

```yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: <yourname>-model
spec:
  name: <yourname>-model-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - name: dummy-classifier
          image: fluescher/sustaino-dummy-service
        - name: real-classifier
          image: gcr.io/academy-300812/<yourname>/eligibility-service:latest
        - name: combiner
          image: fluescher/seldon-combiner
    graph:
      name: combiner
      type: COMBINER
      endpoint:
        type: REST
      children:
      - children: []
        name: dummy-classifier
        type: MODEL
        endpoint:
          type: REST
      - children: []
        name: real-classifier
        type: MODEL
        endpoint:
          type: REST
    name: eligibility
    labels:
      version: v1
    replicas: 1

```

Apply this file 
`kubectl apply -f combiners-seldon-model.yml`
and wait for all pods to be deployed:
`kubectl get pods`


Try send different values. Can you produce both results 1 and 0?



In [4]:
!wget -O- mlproduction.dsiag.ch/seldon/<yourname>/<yourname>-model/api/v1.0/predictions --post-data '{"data": { "ndarray": [[25000, 189625]]}}' --header='Content-Type:application/json'

/bin/bash: yourname: No such file or directory


Check the logs of the container:
    
You can check the logs of a container using `kubectl logs`.

To do that you need to copy the name of the pod: `kubectl get pods`
    
Execute `kubectl logs <podname> <containername>`

eg. `kubectl logs florian-model-eligibility-0-dummy-classifier-post-process-647md7 real-classifier

## Post Processor

Before you start with this exercise remove the preview deployment: `kubectl delete -f combiners-seldon-model.yml`

Sometimes you don't want to output your model result directly. You can use a Post Processor to do that.

We use a sample Post Processor who does the opposite of the dummy-service: it marks all requests as not eligible. You can have a look at it in `services/support/seldon-postprocess`


Create a new file in the folder pipelines named postprocessor-seldon-model.yml:

```yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: <yourname>-model
spec:
  name: <yourname>-model-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - name: dummy-classifier
          image: fluescher/sustaino-dummy-service
        - name: post-process
          image: fluescher/seldon-postprocess
    graph:
      name: dummy-classifier
      type: MODEL
      endpoint:
        type: REST
      children:
      - children: []
        name: post-process
        type: MODEL
        endpoint:
          type: REST
    name: eligibility
    labels:
      version: v1
    replicas: 1

```

Apply this file using `kubectl apply -f postprocessor-seldon-model.yml` and wait for all pods to be deployed. 

Send some Requests to the backend:



In [2]:
!wget -O- mlproduction.dsiag.ch/seldon/<yourname>/<yourname>-model/api/v1.0/predictions --post-data '{"data": { "ndarray": [[25000, 189625]]}}' --header='Content-Type:application/json'

/bin/bash: yourname: No such file or directory


Check the logs of both containers.